In [ ]:
import torch
import numpy as np

In [ ]:
o = torch.randn(4, 4)

In [ ]:
o

In [ ]:
nump = o.numpy()
nump

In [ ]:
roll_down = np.roll(nump, 1, 0)
roll_down[:][0] = 0
print(roll_down)

In [187]:
np.delete(nump, 0, axis=1)

array([[ 0.        ,  0.        ],
       [ 0.18189748, -0.41627452],
       [ 0.        ,  0.        ]], dtype=float32)

In [212]:
def thinned(tensor, axis_offset=0):
    """
    Accepts a tensor. Returns a clone of that tensor with edge
    values raplced with 0's. Axis offset is used for tensors with
    > 2 dimensions.
    """
    x_axis = 0 + axis_offset
    y_axis = 1 + axis_offset
    
    ndarray = tensor.numpy()
    ndarray = np.delete(ndarray, 0, axis=x_axis)
    ndarray = np.delete(ndarray, 0, axis=y_axis)
    ndarray = np.delete(ndarray, -1, axis=x_axis)
    ndarray = np.delete(ndarray, -1, axis=y_axis)
    
    padding_tuples = [(0, 0) for i in range(axis_offset)] + [(1, 1), (1, 1)]
    
    return torch.from_numpy(np.pad(ndarray, padding_tuples, "constant"))
o = torch.randn(4, 4, 4)
print(o)
thinned(o, 1)

tensor([[[ 1.5201, -2.0614, -0.8570, -0.2831],
         [ 0.2077,  2.1475, -0.8182, -0.6065],
         [-1.2676,  1.3903, -0.4679,  0.5685],
         [ 2.3777, -0.8229, -1.0575, -0.6809]],

        [[ 0.2426, -0.0750, -0.4401,  1.5727],
         [-0.9485,  1.4653,  0.1893,  1.0930],
         [-0.0377,  0.4257,  0.1751,  0.2468],
         [-1.0561, -0.3691, -2.0849,  0.2881]],

        [[ 0.0785, -0.6489, -1.9429, -0.8210],
         [-1.3113,  0.3713, -0.5422,  0.7051],
         [-0.3359, -0.8008,  0.1457, -1.5719],
         [-1.0904,  0.5354, -0.4981, -0.1492]],

        [[-1.8082, -0.3415,  0.1009, -0.1306],
         [ 1.8391,  0.9665, -0.8547, -0.8497],
         [ 0.2031,  0.0934,  0.8377, -1.9042],
         [ 0.9570, -0.7310,  0.0378, -1.8152]]])
[[[ 2.1475134  -0.81821984]
  [ 1.3902589  -0.467919  ]]

 [[ 1.465297    0.18930191]
  [ 0.42566073  0.17510444]]

 [[ 0.3712773  -0.5421596 ]
  [-0.80078125  0.14570518]]

 [[ 0.9664796  -0.8546726 ]
  [ 0.0933779   0.83774704]]]


tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  2.1475, -0.8182,  0.0000],
         [ 0.0000,  1.3903, -0.4679,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  1.4653,  0.1893,  0.0000],
         [ 0.0000,  0.4257,  0.1751,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.3713, -0.5422,  0.0000],
         [ 0.0000, -0.8008,  0.1457,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.9665, -0.8547,  0.0000],
         [ 0.0000,  0.0934,  0.8377,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]])

In [ ]:
def shifted(tensor, distance, axis):
    """
    Accepts a tensor. Returns a clonde of that tensor shifted
    by the given distance along the given axis.
    """
    ndarray = tensor.numpy()
    return torch.from_numpy(np.roll(ndarray, distance, axis))

shifted(s, (1, -1), (0, 1))

In [214]:
def all_shifts(tensor, axis_offset=0):
    
    thinned_tensor = thinned(tensor, axis_offset)
    
    shifts = []
    for offset in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, -1), (-1, 1), (1, 1)]:
        shifts.append(shifted(thinned_tensor, offset, (0 + axis_offset, 1 + axis_offset)))
    
    return torch.stack(shifts)

In [174]:
d = all_shifts(o)
print(o.size())
print(d.size())
print(o.unsqueeze(0) - d)
print(d)

torch.Size([4, 4])
torch.Size([8, 4, 4])
tensor([[[ 0.6503,  2.4986,  1.5485, -0.5211],
         [ 0.3640, -2.0444, -0.7561, -1.9755],
         [ 0.3896,  0.6581, -1.0253,  0.7658],
         [ 2.5350,  0.5250,  0.3425, -0.6042]],

        [[ 0.6503,  1.1124, -0.2329, -0.5211],
         [ 0.3640, -1.3862, -1.7814, -1.9755],
         [ 0.3896,  2.0444,  0.7561,  0.7658],
         [ 2.5350, -0.1331,  1.3678, -0.6042]],

        [[ 0.6503,  1.1124, -0.2329, -0.5211],
         [ 1.7502,  0.3952, -1.7814, -1.9755],
         [-0.2685,  1.6834, -1.0253,  0.7658],
         [ 2.5350,  0.5250,  0.3425, -0.6042]],

        [[ 0.6503,  1.1124, -0.2329, -0.5211],
         [ 0.3640, -1.3862, -0.3952, -0.1941],
         [ 0.3896,  0.6581, -1.6834,  1.7911],
         [ 2.5350,  0.5250,  0.3425, -0.6042]],

        [[ 2.0365,  2.8938, -0.2329, -0.5211],
         [-0.2941, -0.3609, -1.7814, -1.9755],
         [ 0.3896,  0.6581, -1.0253,  0.7658],
         [ 2.5350,  0.5250,  0.3425, -0.6042]],

        [

In [173]:
d.std()
np.exp(d.std())

tensor(1.8301)

In [218]:
def smoothing_loss(tensor):
    """
    Calculates smoothing factor loss according to "Deep Correlations for Texture synthesis"
    """
    print(tensor.size()) # [M, Q]
    axis_offset = tensor.dim() - 2
    
    shifts = all_shifts(tensor, axis_offset)
    print(shifts.size()) # should be [8, M, Q]
    
    differences = tensor.unsqueeze(0) - shifts
    print(differences.size())
    
    diff_sq = differences ** 2
    print(diff_sq.size()) # should be [8, M, Q]
    
    diff_sq_std = diff_sq * (-tensor.std())
    print(diff_sq_std.size()) # should be [8, M, Q]
    
    sum_exp = torch.sum(np.exp(diff_sq_std), dim=0)   
    print(sum_exp.size()) # should be [M, Q]
    
    sum_log = torch.sum(torch.log(sum_exp))
    print(sum_log.size()) # should be []
    
    return 1 / (sum_log * (2 * tensor.std()))
    

smoothing_loss(o)

torch.Size([4, 4, 4])
[[[ 2.1475134  -0.81821984]
  [ 1.3902589  -0.467919  ]]

 [[ 1.465297    0.18930191]
  [ 0.42566073  0.17510444]]

 [[ 0.3712773  -0.5421596 ]
  [-0.80078125  0.14570518]]

 [[ 0.9664796  -0.8546726 ]
  [ 0.0933779   0.83774704]]]
torch.Size([8, 4, 4, 4])
torch.Size([8, 4, 4, 4])
torch.Size([8, 4, 4, 4])
torch.Size([8, 4, 4, 4])
torch.Size([4, 4, 4])
torch.Size([])


tensor(0.0071)

In [216]:
def min_difference(tensor):
    shifts = all_shifts(tensor)
    differences = tensor.unsqueeze(0) - shifts
    return torch.min(differences **2)

min_difference(o)

[[[-0.9485445   1.465297    0.18930191  1.0929642 ]
  [-0.03765534  0.42566073  0.17510444  0.2468475 ]]

 [[-1.311298    0.3712773  -0.5421596   0.7050627 ]
  [-0.33587515 -0.80078125  0.14570518 -1.5718988 ]]]


ValueError: Unable to create correctly shaped tuple from [(1, 1), (1, 1)]

In [217]:
x = torch.randn(4, 4, 4)
smoothing_loss(x)

torch.Size([4, 4, 4])
[[[ 0.6036798  -1.2814437 ]
  [-0.35524014 -0.17107017]]

 [[ 0.06382474 -2.6328886 ]
  [-1.0328299   0.47698137]]

 [[ 0.07336043 -0.26236013]
  [ 0.9766886   1.4315487 ]]

 [[ 2.141053    0.8667802 ]
  [ 0.3639648   0.25670803]]]
torch.Size([8, 4, 4, 4])
torch.Size([8, 4, 4, 4])
torch.Size([8, 4, 4, 4])
torch.Size([8, 4, 4, 4])
torch.Size([4, 4, 4])
torch.Size([])


tensor(0.0065)